In [1]:
from langchain_ollama import ChatOllama
model = ChatOllama(model='gemma3:1b')

In [2]:
from pydantic import BaseModel
from typing import List, Literal

class Answer(BaseModel):
    response: Literal["Yes", "No"]

model_struct = model.with_structured_output(Answer)

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

prompt = """You are given a query / task. If you understand the query / have knowledge about the topic, return a 'Yes'.
If you don't have clarity on the asked topic / query, simply return a 'No'.
"""

model_struct.invoke([SystemMessage(content=prompt), HumanMessage(content = "Do you know about lkjdl")])

Answer(response='No')

In [5]:
from langgraph.graph import MessagesState, START, END, StateGraph
from typing import Literal, List, Annotated
import operator

class State(MessagesState):
    answers: Annotated[List[Literal["Yes", "No"]], operator.add]

In [17]:
from langgraph.types import interrupt


def assistant(state):
    prompt = """You are given a query / task. If you understand the query / have knowledge about the topic, return a 'Yes'.
    If you don't have clarity on the asked topic / query, simply return a 'No'.
    """
    response = model_struct.invoke([SystemMessage(content=prompt), state['messages'][-1]])
    return({'answers':[response.response]})

def check(state) -> Literal['next_node', 'fallback']:
    if(state['answers'][-1] == 'Yes'):
        return "next_node"
    elif(state['answers'][-1] == 'No'):
        return 'fallback'

def fallback(state):
    user_value = input(f"Can you rephrase your question '{state['messages'][-1]}': ")

    prompt = """You are given a query / task. If you understand the query / have knowledge about the topic, return a 'Yes'.
    If you don't have clarity on the asked topic / query, simply return a 'No'.
    """
    response = model_struct.invoke([SystemMessage(content=prompt), user_value])

    return ({'messages': HumanMessage(content=user_value), 'answers': [response.response]})

def next_node(state):
    print("Next Sub-Graph") 
    return ({'messages':AIMessage("Ok let me think hahhaha")})


In [18]:
builder = StateGraph(State)

builder.add_node('assistant', assistant)
builder.add_node('fallback', fallback)
builder.add_node('next_node', next_node)

builder.add_edge(START, 'assistant')
builder.add_conditional_edges('assistant', check)
builder.add_conditional_edges('fallback', check)
builder.add_edge('next_node', END)

graph = builder.compile()

In [19]:
graph.invoke({'messages': HumanMessage(content = "Do you know about aksjdhkj")})

Next Sub-Graph


{'messages': [HumanMessage(content='Do you know about aksjdhkj', additional_kwargs={}, response_metadata={}, id='494fb22f-785b-41d6-bcf5-48232402a946'),
  HumanMessage(content='I meant, Dark Matter', additional_kwargs={}, response_metadata={}, id='a77faa74-e8ce-4267-bd8c-66eb059bd82d'),
  AIMessage(content='Ok let me think hahhaha', additional_kwargs={}, response_metadata={}, id='b9406e3d-34c0-4b7e-86ce-4cc9ab6bc7ff')],
 'answers': ['No', 'Yes']}

In [9]:
graph.invoke({'messages': HumanMessage(content = "Do you know about aksjdhkj")})

{'messages': [HumanMessage(content='Do you know about aksjdhkj', additional_kwargs={}, response_metadata={}, id='8d4395a1-0589-422b-a2c2-f8c5e760e36e')],
 'answers': ['No'],
 '__interrupt__': [Interrupt(value="Can you rephrase your question 'content='Do you know about aksjdhkj' additional_kwargs={} response_metadata={} id='8d4395a1-0589-422b-a2c2-f8c5e760e36e'': ", id='b9082c76b0b112168cb71d63ce923bee')]}

In [10]:
graph.invoke({'messages': HumanMessage(content = "Do you know about Certificate")})

Next Sub-Graph


{'messages': [HumanMessage(content='Do you know about Certificate', additional_kwargs={}, response_metadata={}, id='7b6af0c4-0e80-4541-bec8-d0e857aef6bf'),
  AIMessage(content='Ok let me think hahhaha', additional_kwargs={}, response_metadata={}, id='fd569296-ff03-4599-b5aa-e149efe639ed')],
 'answers': ['Yes']}

### For testing purposes I will remove the interrupt and add a python input
**later ofc**

In [21]:
class Solutions(BaseModel):
    solutions: List[str]

In [37]:
# A structured output node to give all the plans -> In place of next_node

prompt = '''You are a problem solving planner. You are given a query / task by the user. 
You have to break the problem down into smaller problems, and return all the smaller problems. 
Each smaller problem statement should be as small as possible, and all of them combined should totally cover the whole problem.
'''

struct_model = model.with_structured_output(Solutions)

In [24]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Hey why was the existence dark matter proposed?')])

Solutions(solutions=["Dark matter is a proposed form of matter that doesn't interact with light, making it invisible to telescopes.", "Scientists noticed that galaxies rotate faster than expected based on the amount of visible matter they contain. This suggests there's extra gravity holding them together.", "Gravitational lensing – the bending of light from distant objects by gravity – provides evidence that there's more mass in the universe than we can see.", 'Cosmological models predict that the universe should have a specific density of matter and energy. Observations don’t match this predicted density.', "The 'missing' mass can potentially explain these discrepancies and create a more complete picture of the universe's structure and evolution."])

In [26]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Why does angler fish stay deep in ocean?')])

Solutions(solutions=['**Thermoregulation:** Anglerfish have specialized photophores (light-producing organs) that emit a faint glow. This glow attracts prey, which they then eat. The deep ocean is a much cooler environment than the surface, so the anglerfish utilizes this light to attract smaller organisms.', '**Predator Avoidance:** The glowing lure can also act as a warning signal to potential predators, helping them avoid being targeted.', '**Camouflage:** While not a primary reason, the glow can subtly camouflage them against the dark depths of the ocean.', '**Food Source:** Anglerfish feed on small crustaceans and other invertebrates, and the lure helps them locate these resources in the dark abyss.', '**Evolutionary Adaptation:** This is a longer-term evolutionary adaptation that has been shaped over millions of years by natural selection.'])

In [27]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='What is the opposite of optimism')])

Solutions(solutions=['Pessimism', 'Disappointment', 'Doubt', 'Cynicism', 'Negative outlook', 'Melancholy', 'Despair', 'Sadness'])

In [28]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Why have we never witnessed a colossal squid')])

Solutions(solutions=['1. Research History of Squid Behavior:  We need to understand how squid have historically been observed and documented. What species have been documented? When? Where? What were the observed behaviors (feeding, migration, etc.)?  This involves looking at paleontological records and recent sightings (even if they’re debated).', '2. Deep-Sea Exploration & Mapping:  The vast majority of squid species are surface-dwelling.  We need to increase our efforts in deep-sea exploration, particularly utilizing advanced sonar, remotely operated vehicles (ROVs), and potentially even autonomous underwater vehicles (AUVs). Focus should be on areas with high sediment and potential for undiscovered species.', '3. Analyze Historical Data for Anomalous Sightings:  We need to rigorously examine historical records – even those that seem unlikely – for clues.  This includes examining old maps, fishing reports, and maritime logs.  Look for patterns in the timing and location of observati

In [29]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='C4 cycle in plants')])

Solutions(solutions=['1. Understand the basic structure of a C4 cycle: It’s a complex metabolic pathway that converts one organic molecule into another, primarily involving the enzyme RuBisCO and a series of reactions.', '2. Define the initial molecule: Start with the starting material – typically RuBP (Ribulose-1,5-bisphosphate) – which is unstable and readily splits.', '3. Describe the key steps:  The cycle involves several steps: (a) Phosphorylation to create 3-PGA (3-phosphoglycerate), (b) Reduction to form G3P (glyceraldehyde-3-phosphate), (c) Regeneration of RuBP – essentially, the cycle keeps the process going.', '4. Explain the role of RuBisCO: RuBisCO is the crucial enzyme responsible for the initial splitting of RuBP, and it’s a significant rate-limiting step.', '5. Discuss the significance of G3P: G3P is a vital intermediate – it’s used to synthesize glucose and other carbohydrates that the plant needs.', '6. Summarize the overall process: The C4 cycle is a pathway that allo

In [30]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='what is the capital of france')])

Solutions(solutions=['The capital of France is Paris.', 'Paris is the largest city in France and a global center for art, fashion, and culture.'])

## Changed System Prompt
added -> "Each smaller problem statement should be small, and just hints at how to solve it." at the end of system prompt


In [32]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Hey why was the existence dark matter proposed?')])

Solutions(solutions=['It arose from observations of galaxies – they seem to rotate faster than they should based on the visible matter alone.', 'Scientists noticed that galaxies have a large amount of invisible matter – dark matter – holding them together.', 'The theory of gravity, as understood at the time, couldn’t explain these observations – it needed something extra.', 'Einstein’s theory of general relativity predicted gravitational effects, but couldn’t incorporate the observed discrepancies.', 'Researchers started looking for a unifying theory that could explain the observed patterns – dark matter was a strong contender.'])

In [33]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Why does angler fish stay deep in ocean?')])

Solutions(solutions=['They need to avoid predators.', 'They’re adapted to find food.', 'They have a slow metabolism.', 'They prefer cooler temperatures.', 'They’re bioluminescent – it helps them stay hidden.'])

In [34]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='What is the opposite of optimism')])

Solutions(solutions=['pessimism', 'skepticism', 'dismay', 'gloomy', 'negative outlook'])

In [35]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Why have we never witnessed a colossal squid')])

Solutions(solutions=['Consider the environmental conditions that might have favored its existence.', 'Research cephalopod biology and known sightings.', 'Explore potential habitat variations – deep sea, unexplored regions, etc.'])

In [36]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='what is the capital of france')])

Solutions(solutions=['Think about the country known as France.', 'It’s a country in Western Europe.', 'It has a significant historical and cultural importance.', 'It’s located in the region of Southern Europe.'])

**Bruh it actually started giving hints like its a quiz**.

**Adding This** - "*Each smaller problem statement should be as small as possible, and all of them combined should totally cover the whole problem.*" to the system prompt.

In [38]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='what is the capital of france')])

Solutions(solutions=['The capital of France is Paris.'])

In [39]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Why have we never witnessed a colossal squid')])

Solutions(solutions=['1. Research Historical Records:** We need to examine historical accounts, nautical logs, and scientific expeditions from the period when cephalopods were likely to have existed. Focus on areas with extensive ocean exploration and the potential for encountering large creatures.', '2. Analyze Existing Cartography & Mapping:** Examine old maps and charts, particularly those from the 17th and 18th centuries, for any clues about potential squid habitats or movements. Look for patterns in sea currents and navigational routes.', '3. Examine Archaeological Evidence:** Investigate potential sites where large squid fossils might have been found – particularly in areas known for marine sediment deposits and deep-sea exploration.', '4. Consult with Marine Biologists & Oceanographers:**  Bring together experts in deep-sea biology, squid behavior, and ocean currents to brainstorm potential reasons why these massive creatures haven’t been documented.', '5. Examine Deep-Sea Explo

In [40]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Why does angler fish stay deep in ocean?')])

Solutions(solutions=['Angler fish are adapted to thrive in the deep ocean due to several key factors:', '1. **Low Light Conditions:** The deep ocean has very little sunlight, and angler fish rely on bioluminescence for hunting.', '2. **Camouflage:** Their unique appearance – a bioluminescent lure – helps them blend in with the faint light filtering down from the surface, making them harder for prey to see.', '3. **Stable Temperatures:** Deep-sea temperatures are relatively stable, which is beneficial for their metabolism and overall survival.', '4. **Limited Competition:** The deep sea is relatively sparsely populated, reducing competition for resources.', '5. **Evolutionary Adaptation:** Over millions of years, angler fish have evolved to be incredibly successful in this extreme environment.'])

In [41]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='Does the pope shit in the woods - What does this mean?')])

Solutions(solutions=['This is a highly controversial and often-repeated internet meme that originated from a 2016 YouTube video.', 'It refers to a series of bizarre and unsettling occurrences involving a man, identified as Jeffrey Cox, claiming to have defecated in various woods and natural settings.', "The video quickly gained popularity due to its shocking and inexplicable nature. It's become a symbol of internet humor and dark internet culture.", "The core of the meme revolves around Cox's elaborate and often bizarre explanations for the incidents, which are typically nonsensical and rely on elaborate storytelling and theatrical pronouncements.", "The 'joke' isn't about the defecation itself, but rather the process of Cox's performance and the resulting internet frenzy. It's a spectacle of theatrical deception.", "It's considered a classic example of internet trolling and dark humor, often criticized for being disturbing and unsettling.", 'The meme’s longevity is largely due to its 

In [42]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='What is relationship security')])

Solutions(solutions=['Understand the core concept of relationship security – how to protect the connections and data associated with a system or organization.', 'Identify the key components of a relationship, such as data, users, and permissions.', 'Define different types of relationships (e.g., user-to-user, data-to-data, etc.) and their specific security considerations.', 'Recognize common threats and vulnerabilities associated with relationship security (e.g., data breaches, unauthorized access, etc.).', 'Outline best practices for securing relationships, including access control, encryption, and monitoring.', 'Explore different security models (e.g., least privilege, defense in depth) and how they apply to relationship security.', 'Consider the implications of changes to a relationship – how security needs to evolve as the system grows.', 'Discuss the role of data lineage and metadata in maintaining relationship security.'])

In [43]:
struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='How to establish relationship security')])

Solutions(solutions=['1. Identify Stakeholders & Risks: Understand who is affected by the relationship, what are their potential vulnerabilities, and what could go wrong?', '2. Define Security Requirements: Clearly outline what data, systems, and processes need to be protected. This includes data sensitivity and compliance regulations.', '3. Implement Security Controls: Implement technical controls like encryption, access controls, firewalls, intrusion detection, and vulnerability scanning.”, ，“4. Establish Monitoring & Logging: Set up systems to continuously monitor activity for suspicious behavior and log all significant events.', '5. Conduct Risk Assessments: Regularly assess potential threats and vulnerabilities to prioritize security efforts.', '6. Develop Incident Response Plan: Create a plan for responding to security incidents, including procedures for containment, eradication, and recovery.', '7. Training & Awareness: Educate employees on security best practices and potential 

In [55]:
solution = struct_model.invoke([SystemMessage(content=prompt), HumanMessage(content='How to establish relationship security')])

In [59]:
type(solution.solutions)

list

**Here** - We *CAN* figure out multiple different meaning which the question can mean, and get a strucutred output of list, with the different meanings, and then ask which one was meant by the user (more like how normal reasoning models work...)
## This performance is really good for now.
# Will implement this later as an update

### Now actually lets check the solutions made, given the problem statement, and smaller problem snippet. 

### Nah No need. 

In [45]:
# Ohkk lets try it once. 

In [46]:
prompt = """You are a problem solver. Given the main problem statement, and a smaller part of the problem, 
You are to solve the SMALLER part of the problem in max 30 words.
"""

# model - Normal

In [ ]:
human_prompt = f"""Problem Statement: "Why does angler fish stay deep in ocean?"
Smaller Problem: Angler fish are adapted to thrive in the deep ocean due to several key factors
"""

In [48]:
model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='They have bioluminescent lures, efficient metabolism, and specialized skeletal structures to withstand immense pressure.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T17:56:29.0756378Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5598819100, 'load_duration': 4027474600, 'prompt_eval_count': 89, 'prompt_eval_duration': 986312400, 'eval_count': 19, 'eval_duration': 480112800, 'model_name': 'gemma3:1b'}, id='run--13eb2b68-ff82-48fb-a05c-d1e9c7c79335-0', usage_metadata={'input_tokens': 89, 'output_tokens': 19, 'total_tokens': 108})

In [49]:
human_prompt = f"""Problem Statement: "Why does angler fish stay deep in ocean?"
Smaller Problem: 1. **Low Light Conditions:** The deep ocean has very little sunlight, and angler fish rely on bioluminescence for hunting.
"""

model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='Angler fish stay deep because bioluminescence allows them to lure prey in the dark depths.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T17:57:19.7563238Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1057948500, 'load_duration': 384150300, 'prompt_eval_count': 99, 'prompt_eval_duration': 218127000, 'eval_count': 19, 'eval_duration': 451147100, 'model_name': 'gemma3:1b'}, id='run--8883f517-8eb9-42d9-8cc5-d754c5a69161-0', usage_metadata={'input_tokens': 99, 'output_tokens': 19, 'total_tokens': 118})

In [50]:
human_prompt = f"""Problem Statement: "Why does angler fish stay deep in ocean?"
Smaller Problem: 2. **Camouflage:** Their unique appearance – a bioluminescent lure – helps them blend in with the faint light filtering down from the surface, making them harder for prey to see.
"""

model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='Angler fish use bioluminescence to camouflage themselves against the deep-sea environment, effectively hiding from predators and attracting prey.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T17:58:34.6071395Z', 'done': True, 'done_reason': 'stop', 'total_duration': 896895600, 'load_duration': 169799500, 'prompt_eval_count': 111, 'prompt_eval_duration': 301066500, 'eval_count': 25, 'eval_duration': 422118400, 'model_name': 'gemma3:1b'}, id='run--03e3b66c-0f4a-44f3-a982-571d9ec236e3-0', usage_metadata={'input_tokens': 111, 'output_tokens': 25, 'total_tokens': 136})

*Yea sure the angler fish will be solved easily*

In [51]:
human_prompt = f"""Problem Statement: "Why was the existence of dark matter proposed in the first place?"
Smaller Problem: It arose from observations of galaxies – they seem to rotate faster than they should based on the visible matter alone.
"""

model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='The observed rotation speeds of galaxies suggest a missing mass, prompting scientists to consider dark matter as an explanation.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T18:00:21.8563398Z', 'done': True, 'done_reason': 'stop', 'total_duration': 866956800, 'load_duration': 131888900, 'prompt_eval_count': 99, 'prompt_eval_duration': 337370100, 'eval_count': 22, 'eval_duration': 394957700, 'model_name': 'gemma3:1b'}, id='run--3c69d79d-df4d-4fce-9288-11fd44b93a90-0', usage_metadata={'input_tokens': 99, 'output_tokens': 22, 'total_tokens': 121})

*I mean I know this will be solved too*
lets try something easier. 

In [52]:
human_prompt = f"""Problem Statement: "What is the capital of France?"
Smaller Problem: "Capital of France is paris"
"""

model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='Paris is the capital of France.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T18:01:28.0378782Z', 'done': True, 'done_reason': 'stop', 'total_duration': 546631700, 'load_duration': 129452500, 'prompt_eval_count': 78, 'prompt_eval_duration': 266958900, 'eval_count': 8, 'eval_duration': 146078200, 'model_name': 'gemma3:1b'}, id='run--b350ff2f-9aee-4dc0-89f5-80d734762ca0-0', usage_metadata={'input_tokens': 78, 'output_tokens': 8, 'total_tokens': 86})

In [ ]:
human_prompt = f"""Problem Statement: "Why have we never witnessed a colossal squid?"
Smaller Problem: "1. Research Historical Records:** We need to examine historical accounts, nautical logs, and scientific expeditions from the period when cephalopods were likely to have existed. Focus on areas with extensive ocean exploration and the potential for encountering large creatures."
"""

model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='Historical records from the 16th-19th centuries, particularly expeditions to the Pacific and Atlantic, suggest possible sightings of colossal squid.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T18:03:14.4454169Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1111817200, 'load_duration': 129569200, 'prompt_eval_count': 121, 'prompt_eval_duration': 383694600, 'eval_count': 30, 'eval_duration': 593997100, 'model_name': 'gemma3:1b'}, id='run--b6ec174f-12a7-4ff3-92dc-03ce0d9f468f-0', usage_metadata={'input_tokens': 121, 'output_tokens': 30, 'total_tokens': 151})

In [54]:

human_prompt = f"""Problem Statement: "Why have we never witnessed a colossal squid?"
Smaller Problem: "Analyze Existing Cartography & Mapping:** Examine old maps and charts, particularly those from the 17th and 18th centuries, for any clues about potential squid habitats or movements. Look for patterns in sea currents and navigational routes.',"
"""

model.invoke([SystemMessage(content=prompt), HumanMessage(content=human_prompt)])

AIMessage(content='Analyzing historical maps reveals potential squid migration routes and areas, suggesting a possible, elusive giant squid presence.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-07T18:04:56.9734881Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1011418200, 'load_duration': 166557600, 'prompt_eval_count': 123, 'prompt_eval_duration': 373205000, 'eval_count': 21, 'eval_duration': 468170200, 'model_name': 'gemma3:1b'}, id='run--8d1f57b1-a19e-4837-9f6f-ca27c1393b7d-0', usage_metadata={'input_tokens': 123, 'output_tokens': 21, 'total_tokens': 144})

Okkk. Just a slight bit of hallucination. Its fine though. 

mmmm. Good enough. 

and lastly we will use a node to add all the solutions